In [6]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
# import dlib, os
from imutils import face_utils
from imutils.face_utils import FaceAligner
import copy
from PIL import Image    
from torch.autograd import Variable
                                    
from preprocess_img import create_mask, get_bounding_rect

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [7]:
torch.__version__

'1.1.0'

In [8]:
model = torch.load('entire_model.pt',map_location='cpu')
model.eval()

MobileNetV2(
  (features): Sequential(
    (0): ConvBNReLU(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNReLU(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_

In [9]:
# model_ft = models.mobilenet_v2(pretrained=False)
# num_ftrs = model_ft.classifier[1].in_features
# model_ft.classifier[1].out_features = nn.Linear(num_ftrs, 10)


# model_ft.load_state_dict(torch.load('model.pt',map_location='cpu'))
# model_ft.eval()

In [10]:
data_dir = 'dataset1/train/6/10.jpg'
test_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
from PIL import Image    
from torch.autograd import Variable
                                    
def predict_image(image):
    model.eval()
    image = Image.open(image)
    image = image.convert('RGB')
    image_tensor = test_transforms(image)
    image_tensor = image_tensor.unsqueeze_(0)
    input = Variable(image_tensor)
#     input=input.unsqueeze_(0)
    input = image_tensor.to(device)
    outputs = model(input)
    _, preds = torch.max(outputs, 1)
    output = model(input)
    index = output.data.cpu().numpy().argmax()
    return index
predict_image(data_dir)

6

In [21]:
test_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])

def get_emojis():
    emojis_folder = 'emojis/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        print(emoji)
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis


def pytorch_process_image(image):
    img = Image.open(image)
    img = img.convert('RGB')
    image_tensor = test_transforms(img)
    inp = Variable(image_tensor.unsqueeze_(0))
    inp = image_tensor.to(device)
    return inp

def pytorch_predict(model, image):
    processed = pytorch_process_image(image)
    output = model(processed)
    pred_class = output.data.to(device).numpy().argmax()
    return pred_class

def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))



def blend(image, emoji, position):
    x, y, w, h = position      
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image


ModuleNotFoundError: No module named 'dlib'

In [ ]:
SHAPE_PREDICTOR_68 = "shape_predictor_68_face_landmarks.dat"
shape_predictor_68 = dlib.shape_predictor(SHAPE_PREDICTOR_68)

detector = dlib.get_frontal_face_detector()

cam = cv2.VideoCapture(1)
if cam.read()[0]==False:
    cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 640) 
fa = FaceAligner(shape_predictor_68, desiredFaceWidth=250)


def emojify():
    emojis = get_emojis()
    disp_probab, disp_class = 0, 0
    while True:
        img = cam.read()[1]
        img = cv2.flip(img, 1)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)
        if len(faces) > 0:
            for i, face in enumerate(faces):
                shape_68 = shape_predictor_68(img, face)
                shape = face_utils.shape_to_np(shape_68)
                mask = create_mask(shape, img)
                masked = cv2.bitwise_and(gray, mask)
                maskAligned = fa.align(mask, gray, face)
                faceAligned = fa.align(masked, gray, face)
                (x0, y0, x1, y1) = get_bounding_rect(maskAligned)
                faceAligned = faceAligned[y0:y1, x0:x1]
                cv2.imshow('faceAligned', faceAligned)
                (x, y, w, h) = face_utils.rect_to_bb(face)
                cv2.imshow('face #{}'.format(i), img[y:y+h, x:x+w])
                pred_class = pytorch_predict(model, faceAligned)
                img = blend(img, emojis[pred_class], (x, y, w, h))
        cv2.imshow('img', img)
        if cv2.waitKey(1) == ord('q'):
            break
            

emojify()